# <center>Implementing LDA in Python</center>

<center>Dr. W.J.B. Mattingly</center>

<center>Smithsonian Data Science Lab and United States Holocaust Memorial Museum</center>

<center>February 2021</center>

## Key Concepts in this Notebook

## Introduction

## Importing the Required Libraries

In [1]:
#https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#1introduction
import numpy as np
import json
import glob

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy
import spacy
from nltk.corpus import stopwords

#vis
import pyLDAvis
# import pyLDAvis.gensim
import pyLDAvis.gensim_models as gensimvis

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

## Preparing the Data

In [2]:
def load_data(file):
    with open (file, "r", encoding="utf-8") as f:
        data = json.load(f) 
    return (data)

def write_data(file, data):
    with open (file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)


In [3]:
stopwords = stopwords.words("english")

In [4]:
print (stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [5]:
data = load_data("data/ushmm_dn.json")["texts"]

print (data[0][0:90])

 My name David Kochalski. I was born in a small town called , and I was born May 5, 1928. 


In [6]:
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)


lemmatized_texts = lemmatization(data)
print (lemmatized_texts[0][0:90])

name bear small town call bear well very hard work child father mother small mill flour bu


In [7]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_texts)

print (data_words[0][0:20])

['name', 'bear', 'small', 'town', 'call', 'bear', 'well', 'very', 'hard', 'work', 'child', 'father', 'mother', 'small', 'mill', 'flour', 'buckwheat', 'prosperous', 'comfortable', 'go']


In [8]:
id2word = corpora.Dictionary(data_words)

corpus = []
for text in data_words:
    new = id2word.doc2bow(text)
    corpus.append(new)

print (corpus[0][0:20])

word = id2word[[0][:1][0]]
print (word)

[(0, 2), (1, 10), (2, 1), (3, 2), (4, 1), (5, 2), (6, 1), (7, 2), (8, 3), (9, 1), (10, 12), (11, 1), (12, 8), (13, 1), (14, 2), (15, 1), (16, 3), (17, 2), (18, 1), (19, 2)]
able


In [9]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=30,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")


## Vizualizing the Data

In [12]:
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
15     0.322455  0.262360       1        1  24.287624
29     0.281474  0.086781       2        1  21.621281
8      0.384957  0.228869       3        1  12.218405
17     0.328271  0.201022       4        1  11.826729
14     0.358064  0.130658       5        1  11.013950
25     0.391258  0.064280       6        1   9.658630
20     0.302214  0.350137       7        1   7.791813
3      0.417634 -0.121597       8        1   1.417063
18     0.040408 -0.248443       9        1   0.154245
19    -0.146701 -0.033637      10        1   0.004537
2     -0.133987 -0.046015      11        1   0.000306
28    -0.133994 -0.046018      12        1   0.000301
26    -0.133992 -0.046018      13        1   0.000299
9     -0.133998 -0.046020      14        1   0.000298
24    -0.133998 -0.046020      15        1   0.000297
10    -0.133999 -0.046020      16        1   0.000296
1     -0.133999 -0.046020      17        1   0.000296
23    -0.133999 -0.046020      18        1   0.000294
13    -0.134003 -0.046022      19        1   0.000293
0     -0.134004 -0.046022      20        1   0.000283
21    -0.134005 -0.046023      21        1   0.000282
4     -0.134005 -0.046023      22        1   0.000280
6     -0.134006 -0.046023      23        1   0.000278
5     -0.134006 -0.046023      24        1   0.000277
16    -0.134006 -0.046023      25        1   0.000277
12    -0.134006 -0.046023      26        1   0.000277
7     -0.134007 -0.046024      27        1   0.000274
22    -0.134008 -0.046024      28        1   0.000272
11    -0.134008 -0.046024      29        1   0.000271
27    -0.134008 -0.046024      30        1   0.000271, topic_info=       Term         Freq        Total Category  logprob  loglift
683     run  2343.000000  2343.000000  Default  30.0000  30.0000
892    year  3466.000000  3466.000000  Default  29.0000  29.0000
152   child  3169.000000  3169.000000  Default  28.0000  28.0000
133    camp  4522.000000  4522.000000  Default  27.0000  27.0000
803   think  5634.000000  5634.000000  Default  26.0000  26.0000
..      ...          ...          ...      ...      ...      ...
643  really     0.000165  3247.502692  Topic30  -9.6434  -3.9795
485    make     0.000165  5070.609368  Topic30  -9.6434  -4.4251
758   start     0.000165  3697.190471  Topic30  -9.6434  -4.1093
859     war     0.000165  3085.892094  Topic30  -9.6434  -3.9285
498    mean     0.000165  2910.094760  Topic30  -9.6435  -3.8699

[3056 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
5647      2  0.981775  academic
904       1  0.080229    accept
904       2  0.649129    accept
904       3  0.003647    accept
904       4  0.113051    accept
...     ...       ...       ...
892       9  0.006635      year
3960      2  0.080563     zloty
3960      3  0.859342     zloty
3960      4  0.080563     zloty
8529      9  0.905720      zoom

[2225 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[16, 30, 9, 18, 15, 26, 21, 4, 19, 20, 3, 29, 27, 10, 25, 11, 2, 24, 14, 1, 22, 5, 7, 6, 17, 13, 8, 23, 12, 28])